In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.linear_model import LinearRegression
from utils import log_wrapper_for_logged_data, DICT_RC_PARAM

In [ ]:
df = pd.read_csv(
    "../generator/data/cplex/final_results/scaling_with_size/UDG_r3_scaling.csv"
)

In [ ]:
df.drop(df[df["Seed"] > 100].index, inplace=True)

In [ ]:
df["log10 Process TTS"] = np.log10(df["Process TTS"])
df["log2 Process TTS"] = np.log2(df["Process TTS"])
df["N"] = np.round(df["L"] ** 2 * df["Density"]).astype(int)

In [ ]:
# dg = df[df["N"]>750].set_index("N")
frac = 0.1
num = int(frac * (len(df[df["N"] == 39]["Seed"]) - 1))
dg = df
dg.sort_values(by="log2 Process TTS", inplace=True)
x = []
y = []
for n in np.unique(dg["N"]):
    y += list(dg[dg["N"] == n][-num:]["log2 Process TTS"])
    x += list(dg[dg["N"] == n][-num:]["N"])

x = np.array(x)
y = np.array(y)

In [ ]:
lr = LinearRegression()
lr.fit(x.reshape(-1, 1), y)
print(lr.score(x.reshape(-1, 1), y))

In [ ]:
### and if we plot both on the same one

In [ ]:
df_unionjack = pd.read_csv("../generator/data/cplex/run_time_d0.8_UDG_8vCPU.csv")

In [ ]:
df_unionjack["log10 Process TTS"] = np.log10(df_unionjack["Process TTS"])
df_unionjack["log2 Process TTS"] = np.log2(df_unionjack["Process TTS"])
df_unionjack["N"] = np.round(df_unionjack["L"] ** 2 * df_unionjack["Density"]).astype(
    int
)

In [ ]:
df_unionjack.drop(df_unionjack[df_unionjack["N"] > 700].index, inplace=True)
df_unionjack.drop(df_unionjack[df_unionjack["Seed"] > 100].index, inplace=True)

In [ ]:
# dg = df[df["N"]>750].set_index("N")
frac = 0.1
num = int(frac * (len(df_unionjack[df_unionjack["N"] == 39]["Seed"]) - 1))
dg = df_unionjack
dg.sort_values(by="log2 Process TTS", inplace=True)
x_unionjack = []
y_unionjack = []
for n in np.unique(dg["N"]):
    y_unionjack += list(dg[dg["N"] == n][-num:]["log2 Process TTS"])
    x_unionjack += list(dg[dg["N"] == n][-num:]["N"])

x_unionjack = np.array(x_unionjack)
y_unionjack = np.array(y_unionjack)

In [ ]:
lr_unionjack = LinearRegression()
lr_unionjack.fit(x_unionjack.reshape(-1, 1), y_unionjack)
print(lr_unionjack.score(x_unionjack.reshape(-1, 1), y_unionjack))

In [ ]:
### both together

In [ ]:
%matplotlib inline
plt.rcParams.update(DICT_RC_PARAM)
fig, ax = plt.subplots(figsize=(15, 10))
positions = np.unique(df_unionjack["N"])
df_unionjack.boxplot(
    "log10 Process TTS", by="N", ax=ax, positions=positions, widths=35, color="b"
)
ax.plot(
    np.unique(x_unionjack),
    np.log(2)
    / np.log(10)
    * lr_unionjack.predict(np.unique(x_unionjack).reshape(-1, 1)),
    label=r"Radius= $\sqrt 2$, TTS = $O(2^{"
    + str(np.round(lr_unionjack.coef_[0], 4))
    + r"N})$",
    color="b",
)

# label = f"Radius= $\sqrt 2$, TTS = $O({np.round(10**(lr_unionjack.coef_[0]),4)}^N)$", color='b')


df.boxplot(
    "log10 Process TTS", by="N", ax=ax, positions=positions, widths=35, color="g"
)
ax.plot(
    np.unique(x),
    np.log(2) / np.log(10) * lr.predict(np.unique(x).reshape(-1, 1)),
    label=r"Radius= $3$, TTS = $O(2^{" + str(np.round(lr.coef_[0], 4)) + r"N})$",
    color="g",
)

# label = f"Radius=3, TTS = $O(2^{"+str(np.round(lr.coef_[0],4))+r"N})$", color='g')
plt.suptitle("")
plt.title("")
ax.set_title("")
ax.set_ylabel("TTS (seconds)")
ax.set_xlabel("$N$")
ax.set_xscale("linear")
ax.legend()
log_wrapper_for_logged_data(ax, y_axis=True)
plt.savefig("scaling_diff_radius.png")
plt.show()